In [20]:
import sys
print(sys.executable)

D:\hacklabs_assignment\Ans-4\myenv\Scripts\python.exe


In [24]:
import cv2
import datetime
import os
import shutil

In [34]:
# Function to empty a folder
def empty_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

# Check and create or empty 'regular_frames' folder
if not os.path.exists('regular_frames'):
    os.makedirs('regular_frames')
else:
    empty_folder('regular_frames')

# Check and create or empty 'motion_frames' folder
if not os.path.exists('motion_frames'):
    os.makedirs('motion_frames')
else:
    empty_folder('motion_frames')

# Define the RTSP URL
rtsp_url = 'rtsp://192.168.1.2:8554/myvideo'  # Replace with your RTSP UR
regular_interval = 32 # Define time interval (in sec) between capturing regular frames

# Define parameters for motion detection
# fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True) # Background subtractor for motion detection
contour_area_threshold = 800  # Increase this value to detect larger (area wise) movements
wait_time = 8 # Define wait time (in sec) between motion detections (to prevent immediate re-detection of the same motion)

# For specific motion detection
min_height_threshold = 92  # Minimum height threshold
max_height_threshold = 380  # Maximum height threshold
min_width_threshold = 127  # Minimum width threshold
max_width_threshold = 447  # Maximum width threshold

In [35]:
# Setup the RTSP stream on VLC Media Player BEFORE executing this code block
# Instructions to setup stream and run this script are given here: https://github.com/atharva-mohite/Assignment-ML_Level-1

# Connect to the RTSP stream
cap = cv2.VideoCapture(rtsp_url)

# Check if the video capture is opened
if not cap.isOpened():
    print("Error: Unable to connect to the video stream.")
    exit()

# To keep track of the time for routine surveillance logging and motion detection
last_saved_time = datetime.datetime.now()
last_motion_time = datetime.datetime.now()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read frame.")
        break

    # Add timestamp to frame
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, timestamp, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

    frame_m = frame
    # Motion detection
    fgmask = fgbg.apply(frame_m)
    _, fgmask = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False
    for contour in contours:
        # if cv2.contourArea(contour) > contour_area_threshold:  # Minimum area to detect motion
        #     motion_detected = True
        #     x, y, w, h = cv2.boundingRect(contour)
        #     cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        x, y, w, h = cv2.boundingRect(contour)
        if (min_height_threshold <= h <= max_height_threshold) and (min_width_threshold <= w <= max_width_threshold):
            motion_detected = True
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Save frame if motion is detected and enough time has passed
    current_time = datetime.datetime.now()
    if motion_detected and (current_time - last_motion_time).seconds >= wait_time:
        motion_capture_path = f'motion_frames/{timestamp}.jpg'.replace(':', '-')
        cv2.imwrite(motion_capture_path, frame_m)
        print(f"Motion detected and saved frame at {motion_capture_path}")
        last_motion_time = current_time  # Update last motion detection time

    # Save frame every 'regular_interval' sec for routine surveillance logging
    if (current_time - last_saved_time).seconds >= regular_interval:
        routine_capture_path = f'regular_frames/{timestamp}.jpg'.replace(':', '-')
        cv2.imwrite(routine_capture_path, frame)
        last_saved_time = current_time
        print(f"Routine capture saved frame at {routine_capture_path}")

cap.release()

Routine capture saved frame at regular_frames/2024-08-11 02-38-45.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-39-08.jpg
Routine capture saved frame at regular_frames/2024-08-11 02-39-18.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-39-45.jpg
Routine capture saved frame at regular_frames/2024-08-11 02-39-50.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-39-53.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-40-01.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-40-10.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-40-18.jpg
Routine capture saved frame at regular_frames/2024-08-11 02-40-22.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-40-26.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-40-34.jpg
Motion detected and saved frame at motion_frames/2024-08-11 02-40-46.jpg
Routine capture saved frame at regular_frames/2024-08-11 02-40-